In [57]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.externals.six import StringIO  
import pydotplus

#give the location of the file below. I have used mine

df =pd.read_csv("Titanic.csv" , index_col = 0)

In [58]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1st,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.337494,B5,Southampton,2,NaN,"St Louis, MO"
2,1st,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.550003,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON"
3,1st,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
4,1st,0,"Allison, Mr. Hudson Joshua Crei",male,30.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON"
5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [59]:
df['pclass_new']= df['pclass'].replace('1st','1').replace('2nd','2').replace('3rd','3')
df['sex_new']= df['sex'].replace('female','0').replace('male','1')
df['age'] = df['age'].fillna(0)
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,pclass_new,sex_new
1,1st,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.337494,B5,Southampton,2,NaN,"St Louis, MO",1,0
2,1st,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.550003,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON",1,1
3,1st,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON",1,0
4,1st,0,"Allison, Mr. Hudson Joshua Crei",male,30.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON",1,1
5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON",1,0


In [60]:
df.sibsp.unique()

array([0, 1, 2, 3, 4, 5, 8])

In [61]:
df_req=df.drop(columns=['pclass','name','sex','parch','ticket','fare','cabin','embarked','boat','body','home.dest'])

In [62]:
df_req.isna().sum()

survived      0
age           0
sibsp         0
pclass_new    0
sex_new       0
dtype: int64

In [63]:
datatypes = df_req.dtypes
datatypes

survived        int64
age           float64
sibsp           int64
pclass_new     object
sex_new        object
dtype: object

In [64]:
# dividing the data into 80 % training and 20 % test data
x = df_req[['pclass_new','sex_new','age','sibsp']]
x.shape

(1309, 4)

In [105]:
y = df_req['survived'].ravel()
y.shape

(1309,)

In [106]:
from sklearn.utils.validation import column_or_1d

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [107]:
x_train.shape

(1047, 4)

In [108]:
y_train.shape

(1047,)

In [109]:
x_test.shape

(262, 4)

In [110]:
y_test.shape

(262,)

In [111]:
from sklearn.neural_network import MLPClassifier
mlnn = MLPClassifier(solver='adam', alpha=1e-3, hidden_layer_sizes=(8,2), random_state=1, activation = 'logistic', learning_rate_init = 0.1, momentum = 0.92)
mlnn.fit(x_train ,  y_train)

MLPClassifier(activation='logistic', alpha=0.001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(8, 2), learning_rate='constant',
              learning_rate_init=0.1, max_iter=200, momentum=0.92,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [112]:
#finding training & test set accuracy results
#Training data set
from sklearn.metrics import accuracy_score
train_set_prediction = mlnn.predict(x_train)
train_set_results  = accuracy_score(train_set_prediction , y_train)
print("The trainig set result is {}".format(train_set_results * 100))
#Testing data set
mlnn.fit(x_train ,y_train)
test_set_prediction  = mlnn.predict(x_test)
test_set_accuracy = accuracy_score(test_set_prediction , y_test)
print("The test set  result is {}".format(test_set_accuracy * 100))

The trainig set result is 81.56638013371537
The test set  result is 77.86259541984732


In [113]:
from sklearn.metrics import confusion_matrix
cm_train = confusion_matrix(train_set_prediction , y_train)
print(cm_train)

[[576 104]
 [ 89 278]]


In [114]:
print(cm_train.ravel())

[576 104  89 278]


In [115]:
#Prediction on training dataset
True_Positive ,False_Positive , False_Negative , True_Negative = cm_train.ravel()
TPR=True_Positive/(True_Positive+False_Negative)
print("in‐sample percent survivors correctly predicted: " , TPR * 100)
TNR = True_Negative / (True_Negative+ False_Positive)
print("in-sample percent fatalities correctly predicted : " , TNR * 100)

in‐sample percent survivors correctly predicted:  86.61654135338345
in-sample percent fatalities correctly predicted :  72.77486910994764


In [116]:
#Predion on Test dataset
cm_test = confusion_matrix(test_set_prediction , y_test)
print(cm_test)
True_Positive_Test ,False_Positive_Test , False_Negative_Test , True_Negative_Test = cm_test.ravel()
TPR_Test=True_Positive_Test/(True_Positive_Test+False_Negative_Test)
print("out-of-sample percent survivors correctly predicted: " , TPR_Test * 100)
TNR_Test = True_Negative_Test / (True_Negative_Test+ False_Positive_Test)
print("Out-of-sample percent fatalities correctly predicted : " , TNR_Test * 100)

[[126  40]
 [ 18  78]]
out-of-sample percent survivors correctly predicted:  87.5
Out-of-sample percent fatalities correctly predicted :  66.10169491525424


In [117]:
#Changing the hidden layer from(8,2) to (5,2)

In [118]:
from sklearn.neural_network import MLPClassifier
mlnn = MLPClassifier(solver='adam', alpha=1e-3, hidden_layer_sizes=(5,2), random_state=1, activation = 'logistic', learning_rate_init = 0.1, momentum = 0.92)
mlnn.fit(x_train ,  y_train)

MLPClassifier(activation='logistic', alpha=0.001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.1, max_iter=200, momentum=0.92,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [119]:
#finding training & test set accuracy results
#Training data set
from sklearn.metrics import accuracy_score
train_set_prediction1 = mlnn.predict(x_train)
train_set_results1  = accuracy_score(train_set_prediction1 , y_train)
print("The trainig set result is {}".format(train_set_results1 * 100))
#Testing data set
mlnn.fit(x_train ,y_train)
test_set_prediction1  = mlnn.predict(x_test)
test_set_accuracy1 = accuracy_score(test_set_prediction1 , y_test)
print("The test set  result is {}".format(test_set_accuracy1 * 100))

The trainig set result is 82.33046800382043
The test set  result is 75.19083969465649


In [120]:
from sklearn.metrics import confusion_matrix
cm_train1 = confusion_matrix(train_set_prediction1 , y_train)
print(cm_train1)
True_Positive ,False_Positive , False_Negative , True_Negative = cm_train1.ravel()
TPR1=True_Positive/(True_Positive+False_Negative)
print("in‐sample percent survivors correctly predicted: " , TPR1 * 100)
TNR1 = True_Negative / (True_Negative+ False_Positive)
print("in-sample percent fatalities correctly predicted : " , TNR1 * 100)

[[625 145]
 [ 40 237]]
in‐sample percent survivors correctly predicted:  93.98496240601504
in-sample percent fatalities correctly predicted :  62.04188481675392


In [121]:
cm_test1 = confusion_matrix(test_set_prediction1 , y_test)
print(cm_test1)
True_Positive_Test ,False_Positive_Test , False_Negative_Test , True_Negative_Test = cm_test1.ravel()
TPR_Test1=True_Positive_Test/(True_Positive_Test+False_Negative_Test)
print("out-of-sample percent survivors correctly predicted: " , TPR_Test1 * 100)
TNR_Test1 = True_Negative_Test / (True_Negative_Test+ False_Positive_Test)
print("Out-of-sample percent fatalities correctly predicted : " , TNR_Test1 * 100)

[[136  57]
 [  8  61]]
out-of-sample percent survivors correctly predicted:  94.44444444444444
Out-of-sample percent fatalities correctly predicted :  51.69491525423729


In [122]:
print("For Decision Tree from HW3")
print("Accuracy: Training data 89.207258834766")
print("Accuracy: Test Data 75.57251908396947")
print("in‐sample percent survivors correctly predicted:  96.2406015037594")
print("in-sample percent fatalities correctly predicted (on training set) :  76.96335078534031")
print("out‐of‐sample percent survivors correctly predicted (on test set) : 88.19444444444444")
print("out-of‐sample percent fatalities correctly predicted (on test set)  60.16949152542372")

For Decision Tree from HW3
Accuracy: Training data 89.207258834766
Accuracy: Test Data 75.57251908396947
in‐sample percent survivors correctly predicted:  96.2406015037594
in-sample percent fatalities correctly predicted (on training set) :  76.96335078534031
out‐of‐sample percent survivors correctly predicted (on test set) : 88.19444444444444
out-of‐sample percent fatalities correctly predicted (on test set)  60.16949152542372


In [123]:
print("For NN")
print("The trainig set result is {}".format(train_set_results * 100))
print("The test set  result is {}".format(test_set_accuracy * 100))
print("in‐sample percent survivors correctly predicted: " , TPR * 100)
print("in-sample percent fatalities correctly predicted : " , TNR * 100)
print("out-of-sample percent survivors correctly predicted: " , TPR_Test * 100)
print("Out-of-sample percent fatalities correctly predicted : " , TNR_Test * 100)

For NN
The trainig set result is 81.56638013371537
The test set  result is 77.86259541984732
in‐sample percent survivors correctly predicted:  86.61654135338345
in-sample percent fatalities correctly predicted :  72.77486910994764
out-of-sample percent survivors correctly predicted:  87.5
Out-of-sample percent fatalities correctly predicted :  66.10169491525424


In [ ]:
#By comparing all the accuracies and especially the out-of-sample and test data accuracies 
#the Neural Network works better accoridng to my code